In [1]:
import requests
import pandas as pd
import time
from pygame import mixer #for sound
from datetime import datetime, timedelta #for looping over date
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
API_key = 'JTFthLtGYuKFQK47MPvQcrabXJRJPk5nUEgaric4f17oP5VJcHyHqsqCMWXu'

#append to dictipnary 
def app_dictionary(id, list, dictionary):
    if id not in dictionary: #checks if column with key already exists 
        dictionary[id] = [list]
    else:
        dictionary[id].append(list) #if yes, it appends to key value

#running api calls and returning result + pagination
def run_api(api_call):
    try:
        result = requests.get(api_call)
        result = result.json()
    except ValueError:
        print(f"Failed to decode JSON. Please check API call: {api_call}")

    #checks if there is a next page
    #if yes it returns true, if no it returns false
    try:
        has_more = result['pagination']['has_more']
    except:
        has_more = False

    try:
        rate_limit = result['rate_limit']['remaining']
        reset = result['rate_limit']['resets_in_seconds']
    except:
        print('no rate limit available')

    return result, has_more, rate_limit, reset

#function for playing sound
def play_sound():
    mixer.init()
    mixer.music.load('done-for-you.mp3')
    mixer.music.play()

def run_pages(api_first, api_second): 
    results, has_more, rate_limit, reset = run_api(f"{api_first}{api_second}")

    page = 2
    while has_more == True:
        if int(rate_limit) > 1:
            next_results, has_more, rate_limit, reset = run_api(f"{api_first}page={page}&{api_second}")
            results['data'] = results['data'] + next_results['data']
            page += 1
        else:
            print(f'Starting sleep timer for: {reset} seconds...')
            time.sleep(int(reset))
    else:
        pass
        
    return results

### **Load data**
<br>
Need to find which teams have played for which seasons, and then set the time interval to the season duration and only get the data for the correct teams.

In [3]:
teams_df = pd.read_csv('data/teams.csv')
seasons_df = pd.read_csv('data/seasons.csv')
leagues_df = pd.read_csv('data/leagues.csv')

Creating dictionary with seasons start date, end date, id, and teams participating in the season

In [48]:
season_dict = {'id':[], 'starting_at':[], 'ending_at':[]}
for row in seasons_df.index:
    app_dictionary('id', str(seasons_df.iloc[row]['id']), season_dict)
    app_dictionary('starting_at', str(seasons_df.iloc[row]['starting_at']), season_dict)
    app_dictionary('ending_at', seasons_df.iloc[row]['ending_at'], season_dict)
    app_dictionary('teams', [i for i in teams_df.loc[teams_df['season_id'] == int(seasons_df.iloc[row]['id'])]['id']], season_dict)


In [54]:
season_dict = pd.DataFrame.from_dict(season_dict)
season_dict.to_csv('data/seasons_info.csv')
season_dict.head()

,id,starting_at,ending_at,teams
0,19734,2022-08-05,2023-05-28,"[78, 11, 8, 19, 42, 29, 71, 51, 14, 236, 52, 2..."
1,19744,2022-08-05,2023-05-27,"[3317, 1079, 510, 3321, 3543, 68, 82, 3320, 67..."
2,19799,2022-08-12,2023-06-04,"[3477, 3468, 485, 36, 1099, 361, 6827, 645, 23..."
3,19806,2022-08-13,2023-06-11,"[10722, 2930, 397, 522, 1123, 625, 345, 1628, ..."
4,21646,2023-08-11,2024-05-19,"[78, 11, 8, 19, 29, 115, 51, 21, 14, 236, 52, ..."


In [63]:
start_dates = season_dict['starting_at']
end_dates = season_dict['ending_at']
teams_inSeasons = season_dict['teams']
season_id = season_dict['id']

# This allows you to work with all rows at once
for start_date, end_date, teams_inSeason, season_id in zip(start_dates, end_dates, teams_inSeasons, season_id):
    print(start_date, end_date, teams_inSeason, season_id)

2022-08-05 2023-05-28 [78, 11, 8, 19, 42, 29, 71, 51, 14, 236, 52, 20, 63, 13, 9, 1, 65, 18, 15, 6] 19734
2022-08-05 2023-05-27 [3317, 1079, 510, 3321, 3543, 68, 82, 3320, 67, 277, 90, 683, 3319, 999, 2726, 366, 794, 503] 19744
2022-08-12 2023-06-04 [3477, 3468, 485, 36, 1099, 361, 6827, 645, 231, 676, 83, 594, 214, 7980, 618, 106, 13258, 528, 459, 377] 19799
2022-08-13 2023-06-11 [10722, 2930, 397, 522, 1123, 625, 345, 1628, 708, 346, 7743, 113, 109, 43, 37, 613, 7790, 8513, 2714, 597] 19806
2023-08-11 2024-05-19 [78, 11, 8, 19, 29, 115, 51, 21, 14, 236, 52, 20, 63, 13, 9, 1, 18, 15, 6, 27] 21646
2023-08-11 2024-05-26 [377, 3477, 3468, 103, 485, 2921, 36, 6827, 645, 231, 676, 83, 2975, 594, 214, 7980, 618, 106, 13258, 459] 21694
2023-08-18 2024-05-18 [1079, 510, 3321, 3543, 68, 82, 3320, 277, 90, 683, 3319, 2831, 999, 2726, 482, 366, 794, 503] 21795
2023-08-19 2024-06-02 [1628, 708, 346, 7743, 113, 102, 109, 43, 37, 4070, 613, 7790, 8513, 2714, 597, 2930, 397, 1123, 625, 585] 21818
20